In [ ]:
import os

from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain



In [ ]:
api_key = os.environ.get("OPENAI_API_KEY")  #please set your OpenAI API key in the .env file or as you prefer

In [ ]:
url = "https://en.wikipedia.org/wiki/Artificial_intelligence"

In [ ]:
loader = WebBaseLoader(url)

In [ ]:
raw_documents = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(raw_documents)  # Split into chunks  

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key= api_key)

In [ ]:
vectorstore = FAISS.from_documents(documents, embeddings)  #storing documents and embeddings in FAISS vector store

In [20]:
memory =  ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [ ]:
qa =  ConversationalRetrievalChain.from_llm(ChatOpenAI(openai_api_key = api_key, model="gpt-5-nano", temperature=1), vectorstore.as_retriever(), memory=memory)

In [ ]:
query = "What is Artificial Intelligence?"

In [ ]:
result = qa.invoke({"question": query})

In [ ]:
result["answer"]